In [77]:
import subprocess
import sys

def install_and_import(*packages):
    for package in packages:
        try:
            # Tenter d'importer chaque bibliothèque
            globals()[package] = __import__(package)
            print(f"Le package '{package}' est déjà installé et importé.")
        except ImportError:
            # Si l'import échoue, installer la bibliothèque
            print(f"Le package '{package}' n'est pas installé. Installation en cours...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            # Ensuite, essayer à nouveau d'importer la bibliothèque
            globals()[package] = __import__(package)
            print(f"Le package '{package}' a été installé et importé.")

# Utilisation de la fonction pour installer et importer plusieurs packages
install_and_import("dask", "pyowm")

Le package 'dask' est déjà installé et importé.
Le package 'pyowm' est déjà installé et importé.


In [78]:
%pip install pyowm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [79]:
from siphon.simplewebservice.ndbc import NDBC
import time
import requests, pandas as pd
from datetime import datetime, timedelta, timezone, time
import os,json
from pathlib import Path
import meteomatics.api as api
import matplotlib.pyplot as plt
import pytz, folium, gspread
from datetime import datetime
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials

In [80]:
# Définir les scopes requis pour accéder à Google Sheets et Google Drive
scopes = [
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive','https://www.googleapis.com/auth/drive.file',
    'https://www.googleapis.com/auth/drive.appdata','https://www.googleapis.com/auth/drive.appfolder'
]

# Chemin vers le fichier de crédentials JSON
path_to_google_cred = "google_sheets_api.json"

# Créer les crédentials à partir du fichier de service account
creds = Credentials.from_service_account_file(path_to_google_cred, scopes=scopes)

# Autoriser l'accès en utilisant les crédentials
client = gspread.authorize(creds)

# ID de la feuille Google Sheets
sheet_id = "1rXyD4WW5DcrO2iCywfwLbqps3Z-RumxQH9hJofhVM24/"

# Ouvrir la feuille Google Sheets en utilisant son ID
spreadsheet = client.open_by_key(sheet_id)

worksheets = spreadsheet.worksheets()

drive_service = build('drive', 'v3', credentials=creds)

Get Data from APIs

Marine API

In [81]:
# Coordonnées de Buoy_51101
latitude = 24.359
longitude = -162.081

# Créer la carte centrée sur les coordonnées
m = folium.Map(location=[latitude, longitude], zoom_start=6)

# Ajouter un marqueur sur Buoy_51101
folium.Marker([latitude, longitude], popup='Buoy_51101').add_to(m)

# Sauvegarder la carte dans un fichier HTML
m.save('buoy_51101_map.html')
m

In [82]:
df_marine = NDBC.realtime_observations('51101')

# Convertir la colonne 'time' en datetime et la garder comme une colonne
df_marine['time'] = pd.to_datetime(df_marine['time'])

# Configurer 'time' comme index, puis resampler
df_marine.set_index('time', inplace=True)

# Resampler à l'heure et prendre la moyenne
df_marine = df_marine.resample('h').mean()

# Optionnel : Si tu veux garder 'time' comme une colonne, on peut le réinitialiser
df_marine.reset_index(inplace=True)

# Afficher le résultat
df_marine.head()

,time,wind_direction,wind_speed,wind_gust,wave_height,dominant_wave_period,average_wave_period,dominant_wave_direction,pressure,air_temperature,water_temperature,dewpoint,visibility,3hr_pressure_tendency,water_level_above_mean
0,2025-01-06 00:00:00+00:00,220.000000,11.166667,14.166667,4.050000,15.0,8.500000,292.000000,1009.333333,25.183333,25.5,NaN,NaN,-2.6,NaN
1,2025-01-06 01:00:00+00:00,210.000000,11.833333,14.833333,4.566667,16.0,9.200000,289.666667,1009.216667,25.300000,25.5,NaN,NaN,-2.3,NaN
2,2025-01-06 02:00:00+00:00,211.666667,12.000000,15.166667,4.100000,16.0,8.533333,284.000000,1009.433333,25.450000,25.5,NaN,NaN,-1.2,NaN
3,2025-01-06 03:00:00+00:00,218.333333,11.833333,15.500000,4.600000,16.0,8.950000,287.500000,1009.916667,25.433333,25.5,NaN,NaN,0.0,NaN
4,2025-01-06 04:00:00+00:00,213.333333,11.500000,14.666667,4.450000,15.0,8.550000,290.000000,1010.316667,25.400000,25.5,NaN,NaN,0.9,NaN


In [83]:
# Arrondir toutes les colonnes numériques (int ou float) à 2 chiffres après la virgule
df_marine = df_marine.applymap(lambda x: round(x, 2) if isinstance(x, (int, float)) else x)

# Vérifier le résultat
df_marine.head()

C:\Users\f.gionnane\AppData\Local\Temp\ipykernel_11508\203937642.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_marine = df_marine.applymap(lambda x: round(x, 2) if isinstance(x, (int, float)) else x)


,time,wind_direction,wind_speed,wind_gust,wave_height,dominant_wave_period,average_wave_period,dominant_wave_direction,pressure,air_temperature,water_temperature,dewpoint,visibility,3hr_pressure_tendency,water_level_above_mean
0,2025-01-06 00:00:00+00:00,220.00,11.17,14.17,4.05,15.0,8.50,292.00,1009.33,25.18,25.5,NaN,NaN,-2.6,NaN
1,2025-01-06 01:00:00+00:00,210.00,11.83,14.83,4.57,16.0,9.20,289.67,1009.22,25.30,25.5,NaN,NaN,-2.3,NaN
2,2025-01-06 02:00:00+00:00,211.67,12.00,15.17,4.10,16.0,8.53,284.00,1009.43,25.45,25.5,NaN,NaN,-1.2,NaN
3,2025-01-06 03:00:00+00:00,218.33,11.83,15.50,4.60,16.0,8.95,287.50,1009.92,25.43,25.5,NaN,NaN,0.0,NaN
4,2025-01-06 04:00:00+00:00,213.33,11.50,14.67,4.45,15.0,8.55,290.00,1010.32,25.40,25.5,NaN,NaN,0.9,NaN


In [84]:
lat = 24.359
lon = -162.081

Meteomatics API

In [85]:
def read_credentials():
    with open('credentials.json', 'r') as file:
        credentials = json.load(file)
    
    meteo_username = credentials.get('username')
    meteo_password = credentials.get('password')
    
    return meteo_username, meteo_password


In [86]:
# Définir les paramètres
parameters = [
        'wind_speed_10m:ms',	
        'wind_dir_10m:d',
        'wind_gusts_10m_1h:ms',
        't_2m:C',
        't_max_2m_24h:C',
        't_min_2m_24h:C',
        'msl_pressure:hPa',
        'precip_1h:mm'
    ]

# Définir les coordonnées
coordinates = [(lat, lon)]

# Définir le modèle
model = 'mix'
# Début URL de l'API
DEFAULT_API_BASE_URL = "https://api.meteomatics.com"
enddate = datetime.now(timezone.utc).replace(minute=0, second=0, microsecond=0)
startdate = enddate - timedelta(days=1)
interval = timedelta(hours=1)

# Fonction de requête API Météo
def meteo_api_request(parameters, coordinates, model, DEFAULT_API_BASE_URL, startdate, enddate, interval):

    # Appel de la fonction read_credentials pour obtenir les informations d'identification
    meteo_username, meteo_password = read_credentials()
    
    # Récupérer les données
    df_meteo = api.query_time_series(coordinates, startdate, enddate, interval, parameters, meteo_username, meteo_password, model=model)
    df_meteo.reset_index(inplace=True)
    return df_meteo

df_meteo=meteo_api_request(parameters, coordinates, model, DEFAULT_API_BASE_URL, startdate, enddate, interval)
df_meteo = df_meteo.rename(columns={'validdate': 'time'})
df_meteo.head()

,lat,lon,time,wind_speed_10m:ms,wind_dir_10m:d,wind_gusts_10m_1h:ms,t_2m:C,t_max_2m_24h:C,t_min_2m_24h:C,msl_pressure:hPa,precip_1h:mm
0,24.359,-162.081,2025-02-19 08:00:00+00:00,5.0,145.7,7.1,24.0,24.0,22.0,1018.8,0.0
1,24.359,-162.081,2025-02-19 09:00:00+00:00,5.0,149.1,6.9,23.9,24.0,22.0,1018.7,0.0
2,24.359,-162.081,2025-02-19 10:00:00+00:00,4.0,154.8,6.9,23.9,24.0,22.0,1018.8,0.0
3,24.359,-162.081,2025-02-19 11:00:00+00:00,4.0,159.1,6.0,23.9,24.0,22.0,1018.6,0.0
4,24.359,-162.081,2025-02-19 12:00:00+00:00,5.0,160.8,6.1,23.8,24.0,22.0,1017.9,0.0


In [87]:
df_marine.head()

,time,wind_direction,wind_speed,wind_gust,wave_height,dominant_wave_period,average_wave_period,dominant_wave_direction,pressure,air_temperature,water_temperature,dewpoint,visibility,3hr_pressure_tendency,water_level_above_mean
0,2025-01-06 00:00:00+00:00,220.00,11.17,14.17,4.05,15.0,8.50,292.00,1009.33,25.18,25.5,NaN,NaN,-2.6,NaN
1,2025-01-06 01:00:00+00:00,210.00,11.83,14.83,4.57,16.0,9.20,289.67,1009.22,25.30,25.5,NaN,NaN,-2.3,NaN
2,2025-01-06 02:00:00+00:00,211.67,12.00,15.17,4.10,16.0,8.53,284.00,1009.43,25.45,25.5,NaN,NaN,-1.2,NaN
3,2025-01-06 03:00:00+00:00,218.33,11.83,15.50,4.60,16.0,8.95,287.50,1009.92,25.43,25.5,NaN,NaN,0.0,NaN
4,2025-01-06 04:00:00+00:00,213.33,11.50,14.67,4.45,15.0,8.55,290.00,1010.32,25.40,25.5,NaN,NaN,0.9,NaN


Check first rows

In [88]:
def convert_to_datetime(value):
    try:
        # Spécifier dayfirst=True pour éviter le warning avec le format %d/%m/%Y
        return pd.to_datetime(value, errors='coerce', dayfirst=True).strftime('%d/%m/%Y-%H:%M') if pd.notna(value) else None
    except Exception as e:
        return None  # Retourner None si la conversion échoue


# Appliquer la fonction sur la colonne 'time' avec apply
df_marine['time'] = df_marine['time'].apply(convert_to_datetime)
df_meteo['time'] = df_meteo['time'].apply(convert_to_datetime)

Rename Columns + Delete Useless Ones

In [89]:
try:
    # Renommage des colonnes pour df_marine
    rename_cols = {
        'wind_direction': 'Wind Dir (°)', 'wind_speed': 'Wind Speed (ms)',
        'wind_gust': 'Wind Gusts 1h (ms)', 'wave_height': 'Wave Height (m)',
        'dominant_wave_period': 'Dominant Wave Period (s)', 'average_wave_period': 'Avg Wave Period (s)',
        'dominant_wave_direction': 'Dominant Wave Direction', 'air_temperature': 'Air Temperature',
        'water_temperature': 'Water Temperature', 'time': 'Datetime', 'pressure': 'Pressure'
    }
    
    # Renommer avec vérif
    for old_col, new_col in rename_cols.items():
        if old_col in df_marine.columns:
            df_marine.rename(columns={old_col: new_col}, inplace=True)
        else:
            print(f"Colonne non trouvée pour renommage : {old_col}")

    # Supprimer les colonnes inutiles
    for col in ['dewpoint', 'visibility', '3hr_pressure_tendency', 'water_level_above_mean', 'lat', 'lon']:
        if col in df_marine.columns:
            df_marine.drop(columns=col, inplace=True)
        else:
            print(f"Colonne non trouvée pour suppression : {col}")

    # Pour df_meteo
    for col in ['lat', 'lon']:
        if col in df_meteo.columns:
            df_meteo.drop(columns=col, inplace=True)
        else:
            print(f"Colonne non trouvée dans df_meteo : {col}")

    # Renommer 'time' en 'Datetime' dans df_meteo
    if 'time' in df_meteo.columns:
        df_meteo.rename(columns={'time': 'Datetime'}, inplace=True)
    else:
        print("Colonne 'time' non trouvée dans df_meteo pour renommage.")

except KeyError as e:
    print(f"Erreur de clé : {e}")


Colonne non trouvée pour suppression : lat
Colonne non trouvée pour suppression : lon


In [90]:
df_meteo.head()

,Datetime,wind_speed_10m:ms,wind_dir_10m:d,wind_gusts_10m_1h:ms,t_2m:C,t_max_2m_24h:C,t_min_2m_24h:C,msl_pressure:hPa,precip_1h:mm
0,19/02/2025-08:00,5.0,145.7,7.1,24.0,24.0,22.0,1018.8,0.0
1,19/02/2025-09:00,5.0,149.1,6.9,23.9,24.0,22.0,1018.7,0.0
2,19/02/2025-10:00,4.0,154.8,6.9,23.9,24.0,22.0,1018.8,0.0
3,19/02/2025-11:00,4.0,159.1,6.0,23.9,24.0,22.0,1018.6,0.0
4,19/02/2025-12:00,5.0,160.8,6.1,23.8,24.0,22.0,1017.9,0.0


In [91]:
print(f"Marine Data Rows Amount : {df_marine.shape[0]}")
marine_null_counts = df_marine.isnull().sum()

print(f"Marine Data Null Values Amount: \n{marine_null_counts}")


Marine Data Rows Amount : 1089
Marine Data Null Values Amount: 
Datetime                    0
Wind Dir (°)                0
Wind Speed (ms)             0
Wind Gusts 1h (ms)          0
Wave Height (m)             8
Dominant Wave Period (s)    8
Avg Wave Period (s)         8
Dominant Wave Direction     8
Pressure                    0
Air Temperature             0
Water Temperature           0
dtype: int64


In [92]:
def handle_null_values(df):
    # Itérer sur les colonnes du DataFrame
    for column in df.columns:
        # Si la colonne est numérique, on remplace les valeurs nulles par la médiane
        if pd.api.types.is_numeric_dtype(df[column]):
            df[column] = df[column].fillna(df[column].median())  # Remplacer sans inplace=True
        # Sinon, on supprime les lignes où il y a des valeurs nulles dans cette colonne
        else:
            df.dropna(subset=[column], inplace=True)
    
    return df  # Retourner le DataFrame après traitement

# Exemple d'utilisation
df_marine = handle_null_values(df_marine)
df_meteo = handle_null_values(df_meteo)

# Vérifier le résultat
df_marine.isnull().sum() # Afficher les valeurs nulles restantes
df_meteo.isnull().sum() # Afficher les valeurs nulles restantes

Datetime                0
wind_speed_10m:ms       0
wind_dir_10m:d          0
wind_gusts_10m_1h:ms    0
t_2m:C                  0
t_max_2m_24h:C          0
t_min_2m_24h:C          0
msl_pressure:hPa        0
precip_1h:mm            0
dtype: int64

In [93]:
df_meteo.head()

,Datetime,wind_speed_10m:ms,wind_dir_10m:d,wind_gusts_10m_1h:ms,t_2m:C,t_max_2m_24h:C,t_min_2m_24h:C,msl_pressure:hPa,precip_1h:mm
0,19/02/2025-08:00,5.0,145.7,7.1,24.0,24.0,22.0,1018.8,0.0
1,19/02/2025-09:00,5.0,149.1,6.9,23.9,24.0,22.0,1018.7,0.0
2,19/02/2025-10:00,4.0,154.8,6.9,23.9,24.0,22.0,1018.8,0.0
3,19/02/2025-11:00,4.0,159.1,6.0,23.9,24.0,22.0,1018.6,0.0
4,19/02/2025-12:00,5.0,160.8,6.1,23.8,24.0,22.0,1017.9,0.0


In [94]:
print(f"Number of Rows in marine DataFrame : {df_marine.shape[0]}")
print(f"Number of Rows in meteo DataFrame : {df_meteo.shape[0]}")

Number of Rows in marine DataFrame : 1089
Number of Rows in meteo DataFrame : 25


Merging Dataframes

In [108]:
intersection_df = pd.merge(df_marine, df_meteo)
intersection_df.head()

,Datetime,Wind Dir (°),Wind Speed (ms),Wind Gusts 1h (ms),Wave Height (m),Dominant Wave Period (s),Avg Wave Period (s),Dominant Wave Direction,Pressure,Air Temperature,Water Temperature,wind_speed_10m:ms,wind_dir_10m:d,wind_gusts_10m_1h:ms,t_2m:C,t_max_2m_24h:C,t_min_2m_24h:C,msl_pressure:hPa,precip_1h:mm
0,19/02/2025-08:00,146.67,5.00,7.33,2.95,15.5,9.70,299.00,1019.20,24.00,24.18,5.0,145.7,7.1,24.0,24.0,22.0,1018.8,0.0
1,19/02/2025-09:00,150.00,4.50,6.50,2.65,16.0,9.20,303.00,1019.30,23.90,24.10,5.0,149.1,6.9,23.9,24.0,22.0,1018.7,0.0
2,19/02/2025-10:00,150.00,3.67,5.50,2.80,15.0,9.67,309.33,1019.37,23.90,24.10,4.0,154.8,6.9,23.9,24.0,22.0,1018.8,0.0
3,19/02/2025-11:00,155.00,4.00,6.00,2.43,15.5,9.33,311.33,1018.93,23.88,24.10,4.0,159.1,6.0,23.9,24.0,22.0,1018.6,0.0
4,19/02/2025-12:00,170.00,5.00,6.83,2.80,15.0,10.15,309.50,1018.43,23.78,24.10,5.0,160.8,6.1,23.8,24.0,22.0,1017.9,0.0


In [96]:
# Effectuer le merge entre df_marine et df_meteo sur la colonne 'time'
df_result = pd.merge(df_marine, df_meteo, on='Datetime', how='right')  # 'inner' pour ne garder que les lignes avec des 'time' correspondants

# Afficher le nombre de lignes de la DataFrame résultante
num_rows = df_result.shape[0]
print(f"Nombre de lignes dans la DataFrame résultante : {num_rows}")


Nombre de lignes dans la DataFrame résultante : 25


In [ ]:
columns_to_drop = ['lat', 'lon']

def drop_columns(df, columns_to_drop):
    for col in columns_to_drop:
        if col in df.columns:
            df = df.drop(columns=col)
        else:
            print(f"'{col}' is not found in the DataFrame")  # Affiche au lieu de retourner une chaîne
    return df  # Toujours retourner le DataFrame

df = drop_columns(df_result, columns_to_drop)

df.head()


In [107]:
# Convertir 'Datetime' en format datetime
df['Datetime'] = pd.to_datetime(df['Datetime'], format='%d/%m/%Y-%H:%M')

# Définir une fonction pour classifier les périodes de la journée
def get_day_period(hour):
    if 5 <= hour < 12:
        return 'Matin'
    elif 12 <= hour < 17:
        return 'Midi'
    elif 17 <= hour < 21:
        return 'Soir'
    else:
        return 'Nuit'

# Appliquer la fonction et créer la nouvelle colonne
df['Day Period'] = df['Datetime'].dt.hour.apply(get_day_period)

df = drop_columns(df, ['Datetime'])
# Voir le résultat
df.head()

,Wind Dir (°),Wind Speed (ms),Wind Gusts 1h (ms),Wave Height (m),Dominant Wave Period (s),Avg Wave Period (s),Dominant Wave Direction,Pressure,Air Temperature,Water Temperature,wind_speed_10m:ms,wind_dir_10m:d,wind_gusts_10m_1h:ms,t_2m:C,t_max_2m_24h:C,t_min_2m_24h:C,msl_pressure:hPa,precip_1h:mm,Day Period
0,146.67,5.00,7.33,2.95,15.5,9.70,299.00,1019.20,24.00,24.18,5.0,145.7,7.1,24.0,24.0,22.0,1018.8,0.0,Matin
1,150.00,4.50,6.50,2.65,16.0,9.20,303.00,1019.30,23.90,24.10,5.0,149.1,6.9,23.9,24.0,22.0,1018.7,0.0,Matin
2,150.00,3.67,5.50,2.80,15.0,9.67,309.33,1019.37,23.90,24.10,4.0,154.8,6.9,23.9,24.0,22.0,1018.8,0.0,Matin
3,155.00,4.00,6.00,2.43,15.5,9.33,311.33,1018.93,23.88,24.10,4.0,159.1,6.0,23.9,24.0,22.0,1018.6,0.0,Matin
4,170.00,5.00,6.83,2.80,15.0,10.15,309.50,1018.43,23.78,24.10,5.0,160.8,6.1,23.8,24.0,22.0,1017.9,0.0,Midi


In [110]:
m = folium.Map(location=[latitude, longitude], zoom_start=6)

folium.Marker([latitude, longitude], popup='Buoy_51101').add_to(m)

m.save('buoy_51101_map.html')

m

In [ ]:
marine_data_sheet = spreadsheet.get_worksheet(0)  # Accéder à la première feuille (gid=0)
marine_data = marine_data_sheet.get_all_records()  # Récupérer toutes les données de la feuille
print(marine_data)  # Afficher les données

In [111]:
from main import addition

r = addition(3,5)

r

ImportError: cannot import name 'addition' from 'main' (c:\Users\f.gionnane\Downloads\Projects\Oceanography-Project-WIP\main.py)